In [230]:
import pandas as pd 
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split

train = pd.read_csv('E:/personal_projects/competitive-data-science-predict-future-sales/sales_train.csv')



In [231]:
shops = pd.read_csv('E:/personal_projects/competitive-data-science-predict-future-sales/shops.csv')

In [232]:
items = pd.read_csv('E:/personal_projects/competitive-data-science-predict-future-sales/items.csv')


In [233]:
categories = pd.read_csv('E:/personal_projects/competitive-data-science-predict-future-sales/item_categories.csv')


In [234]:
test = pd.read_csv('E:/personal_projects/competitive-data-science-predict-future-sales/test.csv')


In [235]:
train['date'] = pd.to_datetime(train['date'], format = "%d.%m.%Y")

In [236]:
train['year'] = train['date'].dt.year

In [237]:
train['month'] = train['date'].dt.month

In [238]:
train.drop(['date'], inplace = True , axis = 1)

In [239]:
train = train.groupby([c for c in train.columns if c not in ['item_cnt_day'] ], as_index = False)['item_cnt_day'].sum()

In [240]:
train = train.rename(columns = {'item_cnt_day' : 'item_cnt_month'})

In [241]:
## mean of sales per product per shop
train_mean_month = train [['shop_id','item_id','item_cnt_month']].groupby(['shop_id','item_id'], as_index=False)[['item_cnt_month']].mean()


In [242]:
train_mean_month= train_mean_month.rename(columns = {'item_cnt_month' : 'item_mean_month'})

In [243]:
train= pd.merge(train , train_mean_month , how = 'left' ,on = ['shop_id' ,'item_id'] )

In [244]:
shop_prev_month33 = train [train['date_block_num'] == 33 ][['shop_id' ,'item_id' ,'date_block_num','item_cnt_month'] ]


In [245]:
shop_prev_month33 = shop_prev_month33.groupby(['item_id' , 'shop_id'] , as_index= False)['item_cnt_month'].sum()

In [246]:
shop_prev_month33 = shop_prev_month33.rename(columns = {'item_cnt_month' : 'item_cnt_prev_month'})


In [247]:
shop_prev_month = train.groupby(['item_id' , 'shop_id' ,'date_block_num'] , as_index= False)['item_cnt_month'].sum()

In [248]:
shop_prev_month = shop_prev_month.rename(columns = {'item_cnt_month' : 'item_cnt_prev_month'})



In [249]:
shop_prev_month

,item_id,shop_id,date_block_num,item_cnt_prev_month
0,0,54,20,1.0
1,1,55,15,2.0
2,1,55,18,1.0
3,1,55,19,1.0
4,1,55,20,1.0
...,...,...,...,...
1609119,22168,12,8,1.0
1609120,22168,16,1,1.0
1609121,22168,42,1,1.0
1609122,22168,43,2,1.0


In [250]:
shop_prev_month['date_block_num']=(shop_prev_month['date_block_num'] + 1)

In [251]:
shop_prev_month

,item_id,shop_id,date_block_num,item_cnt_prev_month
0,0,54,21,1.0
1,1,55,16,2.0
2,1,55,19,1.0
3,1,55,20,1.0
4,1,55,21,1.0
...,...,...,...,...
1609119,22168,12,9,1.0
1609120,22168,16,2,1.0
1609121,22168,42,2,1.0
1609122,22168,43,3,1.0


In [252]:
t = train

In [253]:
trainn= pd.merge(train , shop_prev_month , how = 'left' ,on = ['shop_id' ,'item_id' ,'date_block_num'] )


In [254]:
trainn

,date_block_num,shop_id,item_id,item_price,year,month,item_cnt_month,item_mean_month,item_cnt_prev_month
0,0,0,32,221.0,2013,1,6.0,8.000000,NaN
1,0,0,33,347.0,2013,1,3.0,3.000000,NaN
2,0,0,35,247.0,2013,1,1.0,7.500000,NaN
3,0,0,43,221.0,2013,1,1.0,1.000000,NaN
4,0,0,51,127.0,2013,1,1.0,1.666667,NaN
...,...,...,...,...,...,...,...,...,...
1739017,33,59,22087,119.0,2015,10,6.0,6.235294,3.0
1739018,33,59,22088,119.0,2015,10,2.0,6.000000,1.0
1739019,33,59,22091,179.0,2015,10,1.0,2.294118,3.0
1739020,33,59,22100,629.0,2015,10,1.0,1.000000,1.0


In [255]:
trainn = trainn [trainn['date_block_num']>0]

In [256]:
trainn['item_cnt_prev_month'].isna().sum()

911313

In [257]:
trainn = trainn.fillna(0.)
trainn.shape

(1671900, 9)

In [258]:
trainn =  pd.merge (trainn ,  items , how  = 'left',on = ['item_id'])
trainn =  pd.merge (trainn ,  categories , how  = 'left',on = ['item_category_id'])
trainn =  pd.merge (trainn,  shops , how  = 'left',on = ['shop_id'])

In [259]:
test['month'] = 11 
test = pd.merge (test ,  items , how  = 'left',on = ['item_id'])
test =  pd.merge (test,  categories , how  = 'left',on = ['item_category_id'])
test = pd.merge ( test,  shops , how  = 'left',on = ['shop_id'])

In [260]:
test['year'] = 2015
test ['date_block_num'] = 34
test= pd.merge(test, shop_prev_month33, how = 'left' ,on = ['item_id' , 'shop_id'] ).fillna(0.)
test= pd.merge(test , train_mean_month , how = 'left' ,on = ['item_id' , 'shop_id'] )




In [261]:
test

,ID,shop_id,item_id,month,item_name,item_category_id,item_category_name,shop_name,year,date_block_num,item_cnt_prev_month,item_mean_month
0,0,5,5037,11,"NHL 15 [PS3, русские субтитры]",19,Игры - PS3,"Вологда ТРЦ ""Мармелад""",2015,34,0.0,1.181818
1,1,5,5320,11,ONE DIRECTION Made In The A.M.,55,Музыка - CD локального производства,"Вологда ТРЦ ""Мармелад""",2015,34,0.0,NaN
2,2,5,5233,11,"Need for Speed Rivals (Essentials) [PS3, русск...",19,Игры - PS3,"Вологда ТРЦ ""Мармелад""",2015,34,1.0,1.428571
3,3,5,5232,11,"Need for Speed Rivals (Classics) [Xbox 360, ру...",23,Игры - XBOX 360,"Вологда ТРЦ ""Мармелад""",2015,34,0.0,1.000000
4,4,5,5268,11,"Need for Speed [PS4, русская версия]",20,Игры - PS4,"Вологда ТРЦ ""Мармелад""",2015,34,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
214195,214195,45,18454,11,СБ. Союз 55,55,Музыка - CD локального производства,"Самара ТЦ ""ПаркХаус""",2015,34,1.0,1.800000
214196,214196,45,16188,11,Настольная игра Нано Кёрлинг,64,Подарки - Настольные игры,"Самара ТЦ ""ПаркХаус""",2015,34,0.0,NaN
214197,214197,45,15757,11,НОВИКОВ АЛЕКСАНДР Новая коллекция,55,Музыка - CD локального производства,"Самара ТЦ ""ПаркХаус""",2015,34,0.0,1.166667
214198,214198,45,19648,11,ТЕРЕМ - ТЕРЕМОК сб.м/ф (Регион),40,Кино - DVD,"Самара ТЦ ""ПаркХаус""",2015,34,0.0,NaN


In [262]:
from sklearn import preprocessing
for c in ['item_name' , 'item_category_name' , 'shop_name'] :
    e = preprocessing.LabelEncoder()
    e.fit(list(trainn[c].unique()) + list(test[c].unique()))
    trainn[c] = e.transform(trainn[c].astype(str))
    test[c] = e.transform(test[c].astype(str))
    print(c)

item_name
item_category_name
shop_name


In [263]:
trainn.drop(['item_price'] , axis = 1 , inplace = True)

In [264]:
from sklearn.model_selection import train_test_split
import numpy as np
X_train = trainn [trainn['date_block_num']<33]
y_train = trainn [trainn['date_block_num']<33] ['item_cnt_month']
X_test = trainn [trainn['date_block_num']==33]
y_test = X_test['item_cnt_month'].clip(0. , 20.)


In [265]:
X_train = X_train.drop(['item_cnt_month'], axis = 1)
X_test = X_test.drop(['item_cnt_month'], axis = 1)


In [266]:
from sklearn.ensemble import RandomForestRegressor , GradientBoostingRegressor
params = {'n_estimators': 50, 'max_depth': 6, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}
clf = GradientBoostingRegressor(**params)
clf.fit(X_train, y_train)


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.01, loss='ls', max_depth=6,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=50,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [271]:
import sklearn
from sklearn.metrics import mean_squared_error
mse = sklearn.metrics.mean_absolute_error(y_test, clf.predict(X_test))
print("MSE: %.4f" % mse)

MSE: 1.0888


In [163]:
test = test.fillna(0.)

In [164]:
su = pd.DataFrame()


In [168]:
test

,shop_id,item_id,month,item_name,item_category_id,item_category_name,shop_name,year,date_block_num,item_cnt_prev_month,item_mean_month
0,5,5037,11,4949,19,19,5,2015,34,0.0,1.181818
1,5,5320,11,5227,55,55,5,2015,34,0.0,0.000000
2,5,5233,11,5142,19,19,5,2015,34,1.0,1.428571
3,5,5232,11,5141,23,23,5,2015,34,0.0,1.000000
4,5,5268,11,5177,20,20,5,2015,34,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
214195,45,18454,11,18169,55,55,45,2015,34,1.0,1.800000
214196,45,16188,11,15930,64,64,45,2015,34,0.0,0.000000
214197,45,15757,11,15503,55,55,45,2015,34,0.0,1.166667
214198,45,19648,11,19349,40,40,45,2015,34,0.0,0.000000


In [194]:
submi = pd.DataFrame()
test2 = pd.read_csv('E:/personal_projects/competitive-data-science-predict-future-sales/test.csv')

submi['ID'] = test2.ID


In [195]:
submi['item_cnt_month'] = clf.predict(test)
submi['item_cnt_month'] = (submi['item_cnt_month'])
submi.to_csv('subminnn.csv', index = False)

In [196]:
submi

,ID,item_cnt_month
0,0,2.857176
1,1,2.833671
2,2,2.857176
3,3,2.857176
4,4,2.857176
...,...,...
214195,214195,18.211353
214196,214196,18.211353
214197,214197,18.211353
214198,214198,16.568421
